In [1]:
import findspark
findspark.init()


In [2]:
import pyspark 
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("StudyApp").getOrCreate()

In [4]:
df=spark.read.csv("D:\Height-Weight1.csv",header=True,inferSchema=True)
df.show()

+---+------+------+-----+-----+
|sex|weight|height|repwt|repht|
+---+------+------+-----+-----+
|  F|    39|   157|   41|  153|
|  F|    43|   154| null| null|
|  F|    44|   157|   44|  155|
|  F|    45|   163|   45|  160|
|  F|    45|   157|   45|  153|
|  F|    47|   163|   47|  160|
|  M|    47|   153| null|  154|
|  F|    47|   162|   47|  160|
|  M|    47|   150|   45|  152|
|  M|    48|   163|   44|  160|
|  M|    49|   161| null| null|
|  F|    50|   166|   50|  165|
|  F|    50|   166|   50|  161|
|  F|    50|   171| null| null|
|  F|    50|   160|   55|  150|
|  F|    50|   169|   50|  165|
|  F|    50|   148|   47|  148|
|  F|    50|   158|   49|  155|
|  F|    51|   161|   52|  158|
|  F|    51|   163|   50|  160|
+---+------+------+-----+-----+
only showing top 20 rows



In [6]:
df.printSchema()

root
 |-- sex: string (nullable = true)
 |-- weight: integer (nullable = true)
 |-- height: integer (nullable = true)
 |-- repwt: integer (nullable = true)
 |-- repht: integer (nullable = true)



In [8]:
#formating
from pyspark.ml.feature import VectorAssembler 
featureassembler=VectorAssembler(inputCols=['height'],outputCol='Heightdata')
df1=featureassembler.transform(df)
df1.show()


+---+------+------+-----+-----+----------+
|sex|weight|height|repwt|repht|Heightdata|
+---+------+------+-----+-----+----------+
|  F|    39|   157|   41|  153|   [157.0]|
|  F|    43|   154| null| null|   [154.0]|
|  F|    44|   157|   44|  155|   [157.0]|
|  F|    45|   163|   45|  160|   [163.0]|
|  F|    45|   157|   45|  153|   [157.0]|
|  F|    47|   163|   47|  160|   [163.0]|
|  M|    47|   153| null|  154|   [153.0]|
|  F|    47|   162|   47|  160|   [162.0]|
|  M|    47|   150|   45|  152|   [150.0]|
|  M|    48|   163|   44|  160|   [163.0]|
|  M|    49|   161| null| null|   [161.0]|
|  F|    50|   166|   50|  165|   [166.0]|
|  F|    50|   166|   50|  161|   [166.0]|
|  F|    50|   171| null| null|   [171.0]|
|  F|    50|   160|   55|  150|   [160.0]|
|  F|    50|   169|   50|  165|   [169.0]|
|  F|    50|   148|   47|  148|   [148.0]|
|  F|    50|   158|   49|  155|   [158.0]|
|  F|    51|   161|   52|  158|   [161.0]|
|  F|    51|   163|   50|  160|   [163.0]|
+---+------

In [9]:
df1.printSchema()

root
 |-- sex: string (nullable = true)
 |-- weight: integer (nullable = true)
 |-- height: integer (nullable = true)
 |-- repwt: integer (nullable = true)
 |-- repht: integer (nullable = true)
 |-- Heightdata: vector (nullable = true)



In [10]:
fd=df1.select(['Heightdata','Weight'])
fd.show()
#final data

+----------+------+
|Heightdata|Weight|
+----------+------+
|   [157.0]|    39|
|   [154.0]|    43|
|   [157.0]|    44|
|   [163.0]|    45|
|   [157.0]|    45|
|   [163.0]|    47|
|   [153.0]|    47|
|   [162.0]|    47|
|   [150.0]|    47|
|   [163.0]|    48|
|   [161.0]|    49|
|   [166.0]|    50|
|   [166.0]|    50|
|   [171.0]|    50|
|   [160.0]|    50|
|   [169.0]|    50|
|   [148.0]|    50|
|   [158.0]|    50|
|   [161.0]|    51|
|   [163.0]|    51|
+----------+------+
only showing top 20 rows



In [11]:
#Spliting of data into tranin and test data  
from pyspark.ml.regression import LinearRegression


In [12]:
 train,test=fd.randomSplit([.70,.30])

In [13]:
LR=LinearRegression(featuresCol='Heightdata',labelCol='Weight')

In [14]:
tm=LR.fit(train)

In [15]:
#coefficeint and intercept
print(tm.intercept)
print(tm.coefficients)#coefficient bet weigth and heigth

-122.91150874098571
[1.1024179467308415]


In [16]:
#evaluate the  train data
res=tm.evaluate(test)
res.predictions.show()

+----------+------+------------------+
|Heightdata|Weight|        prediction|
+----------+------+------------------+
|   [148.0]|    50| 40.24634737517883|
|   [154.0]|    43| 46.86085505556389|
|   [161.0]|    53| 54.57778068267976|
|   [161.0]|    54| 54.57778068267976|
|   [162.0]|    47|  55.6801986294106|
|   [162.0]|    53|  55.6801986294106|
|   [162.0]|    60|  55.6801986294106|
|   [163.0]|    45|56.782616576141464|
|   [164.0]|    52|  57.8850345228723|
|   [164.0]|    55|  57.8850345228723|
|   [164.0]|    59|  57.8850345228723|
|   [165.0]|    55| 58.98745246960314|
|   [165.0]|    55| 58.98745246960314|
|   [165.0]|    64| 58.98745246960314|
|   [165.0]|    68| 58.98745246960314|
|   [166.0]|    50| 60.08987041633398|
|   [166.0]|    58| 60.08987041633398|
|   [166.0]|    59| 60.08987041633398|
|   [166.0]|    64| 60.08987041633398|
|   [166.0]|    65| 60.08987041633398|
+----------+------+------------------+
only showing top 20 rows



D:\PythonAnaconda\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [17]:
print("this is R2",res.r2)#Accuracy
print("this is mean absolute error",res.meanAbsoluteError)
print("this is mean square root(RMSE)",res.meanSquaredError)


this is R2 0.6496199761782993
this is mean absolute error 5.89160378773051
this is mean square root(RMSE) 54.127414573580864
